# Collecting Data
I will use the yahoo finance API to get the start and end price for our benchmark, DIA. DIA is the oldest Dow Jones mutual fund. Yahoo finance has data going back to 1998 for this fund. We will get the prices for the components of the Dow Jones circa 1997 (as determined from [this wikipedia article](https://en.wikipedia.org/wiki/Historical_components_of_the_Dow_Jones_Industrial_Average)) for the same time period as our benchmark.

I will keep transformation and cleaning to a minimum during the collection phase.

In [5]:
import datetime as dt
import yfinance as yf
import pandas as pd
import json
import os
from time import sleep

Setup directories if necessary

In [12]:
try:
    os.makedirs("data/raw") 
except FileExistsError as e:
    print(e)

[WinError 183] Cannot create a file when that file already exists: 'data/raw'


## Get DIA History

In [13]:
def max_start_and_end_data_for(symbol):
    df = yf.Ticker(symbol).history(period="max").iloc[[0, -1]]
    return add_symbol_as_top_index(df, symbol)

def add_symbol_as_top_index(df, symbol):
    return pd.concat({f'{symbol}': df}, names=['Symbol'])

This retrieves the start and end data for DIA and saves it locally.

Set `force` to `True` to run.

In [41]:
force = False

if force:
    dia_history = max_start_and_end_data_for("DIA")
    dia_history.to_pickle("data/raw/DIA.pickle")

Loads DIA history

In [6]:
dia_history = pd.read_pickle("data/raw/DIA.pickle")
dia_history

Open        High         Low       Close   Volume  \
Symbol Date                                                                  
DIA    1998-01-20   47.228395   47.854313   46.981821   47.835346  1744600   
       2021-01-14  310.630005  312.268311  310.130005  310.720001  1787230   

                   Dividends  Stock Splits  
Symbol Date                                 
DIA    1998-01-20        0.0             0  
       2021-01-14        0.0             0

## Get History for Dow Companies

In [7]:
def dow_symbols(path):
    with open(f'{path}.json', 'r') as fp:
        symbols = json.load(fp)
    return [list(k.keys())[0] for k in symbols]

def start_and_end_data_for(symbol, start_timestamp, end_timestamp):
    start_end_timestamp = start_timestamp + dt.timedelta(days=1)
    end_end_timestamp = end_timestamp + dt.timedelta(days=1)
    
    start = yf.Ticker(symbol).history(start=start_timestamp, end=start_end_timestamp)
    end = yf.Ticker(symbol).history(start=end_timestamp, end=end_end_timestamp)
    
    return(pd.concat([start, end]))

Here we get the individual stocks in the Dow Jones based according to `data/dow-symbols` and the time period established by our benchmark, DIA and pickle them.

Set `force` to `True` to run.

In [35]:
force = False

if force:
    dow_sym = dow_symbols("data/dow-symbols")
    start_timestamp = dia_history.index[0][1]
    end_timestamp = dia_history.index[1][1]

    for s in dow_sym:
        df = start_and_end_data_for(s, start_timestamp, end_timestamp)
        combined = add_symbol_as_top_index(df, s)
        combined.to_pickle(f"data/raw/{s}.pickle")
        sleep(0.6)

Loads the dataframes from file and combines everything into a single `data/dow.csv`

In [36]:
pickles = [f for f in os.listdir("data/raw")]
dataframes = [pd.read_pickle(f"data/raw/{p}") for p in pickles]
df_combined = pd.concat(dataframes)
df_combined.to_csv("data/dow.csv")

Elsewhere, we can load our data from csv like so:

In [14]:
pd.read_csv("data/dow.csv", index_col=[0,1])

Open        High         Low       Close      Volume  \
Symbol Date                                                                     
AA     1998-01-20   28.716094   29.547440   28.681454   29.530121   2176900.0   
       2021-01-14   25.000000   25.379999   24.850100   25.090000   3948281.0   
AIG    1998-01-20  482.618128  486.123816  476.775276  485.831665    207900.0   
       2021-01-14   41.000000   41.448101   40.680000   41.299999   3873888.0   
AXP    1998-01-20   17.604445   17.765462   17.537355   17.698372   4635200.0   
...                       ...         ...         ...         ...         ...   
VZ     2021-01-14   56.889999   57.424999   56.660000   57.040001  17987541.0   
WMT    1998-01-20   13.316331   14.115731   13.316331   14.052621   7682800.0   
       2021-01-14  147.820007  148.000000  146.035004  146.970001   6749975.0   
XOM    1998-01-20   15.468195   15.611567   15.372614   15.515985   6348000.0   
       2021-01-14   48.500000   51.080002   48.500000   50.310001  34741787.0   

                   Dividends  Stock Splits  Adj Close  
Symbol Date                                            
AA     1998-01-20        0.0           0.0        NaN  
       2021-01-14        0.0           0.0        NaN  
AIG    1998-01-20        0.0           0.0        NaN  
       2021-01-14        0.0           0.0        NaN  
AXP    1998-01-20        0.0           0.0        NaN  
...                      ...           ...        ...  
VZ     2021-01-14        0.0           0.0        NaN  
WMT    1998-01-20        0.0           0.0        NaN  
       2021-01-14        0.0           0.0        NaN  
XOM    1998-01-20        0.0           0.0        NaN  
       2021-01-14        0.0           0.0        NaN  

[62 rows x 8 columns]